# Sentiment Analysis

In [13]:
import torch
import pandas as pd
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [14]:
torch.cuda.is_available()

True

In [15]:
device

device(type='cuda', index=0)

In [16]:
train_df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pointwise_v3_3.pkl')
train_df = train_df.reset_index(drop=True)

In [17]:
train_df = train_df.reset_index(drop=True)

In [18]:
import spacy
import pandas as pd
nlp = spacy.load("en_core_web_md")
from collections import Counter

## SPACY and TextBlob
* [SPACY and TextBlob](https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9)

In [19]:
# Counter(train_df.name)
# Joe Boccardi's Ristorante

In [20]:
sentences = ['Mushrooms n calamari was not bad']

In [21]:
tmp = train_df[train_df.name== 'Joe Boccardi\'s Ristorante']

In [22]:
sentences = list(tmp.text)
sentences = list(dict.fromkeys(sentences)) # deduplicate

In [23]:
sentences

['This is the best italian restaurant andrea makes the best chicken ala mario. The house dressing is to die for.',
 "Cajun pasta was YUM!!!! Mushrooms n calamari wasn't bad... waitress n staff were great...servings big def recommend!!",
 'Still our favorite anytime place, but they make Saturday date night even better with their seafood special linguini and regular chicken parm. Sooo good!!']

In [24]:
aspects = []

for sentence in sentences:
    for sent in sentence.split('.'):
        doc = nlp(sent)
        print(doc)
        descriptive_term = ''
        target = ''
        for token in doc:
            if token.dep_ == 'nsubj' and token.pos_ == 'NOUN':
                target = token.text
            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ != 'ADV':
                        continue
                    prepend += child.text + ' '
                descriptive_term = prepend + token.text
        aspects.append({'aspect': target,'description': descriptive_term})

print(aspects)


This is the best italian restaurant andrea makes the best chicken ala mario
 The house dressing is to die for

Cajun pasta was YUM!!!! Mushrooms n calamari wasn't bad


 waitress n staff were great


servings big def recommend!!
Still our favorite anytime place, but they make Saturday date night even better with their seafood special linguini and regular chicken parm
 Sooo good!!
[{'aspect': '', 'description': 'best'}, {'aspect': 'house', 'description': ''}, {'aspect': '', 'description': ''}, {'aspect': 'Mushrooms', 'description': 'bad'}, {'aspect': '', 'description': ''}, {'aspect': '', 'description': ''}, {'aspect': 'staff', 'description': 'great'}, {'aspect': '', 'description': ''}, {'aspect': '', 'description': ''}, {'aspect': '', 'description': 'big'}, {'aspect': '', 'description': 'regular'}, {'aspect': '', 'description': 'good'}]


In [25]:
from textblob import TextBlob
for aspect in aspects:
    aspect['sentiment'] = TextBlob(aspect['description']).sentiment.polarity
print(aspects)


[{'aspect': '', 'description': 'best', 'sentiment': 1.0}, {'aspect': 'house', 'description': '', 'sentiment': 0.0}, {'aspect': '', 'description': '', 'sentiment': 0.0}, {'aspect': 'Mushrooms', 'description': 'bad', 'sentiment': -0.6999999999999998}, {'aspect': '', 'description': '', 'sentiment': 0.0}, {'aspect': '', 'description': '', 'sentiment': 0.0}, {'aspect': 'staff', 'description': 'great', 'sentiment': 0.8}, {'aspect': '', 'description': '', 'sentiment': 0.0}, {'aspect': '', 'description': '', 'sentiment': 0.0}, {'aspect': '', 'description': 'big', 'sentiment': 0.0}, {'aspect': '', 'description': 'regular', 'sentiment': 0.0}, {'aspect': '', 'description': 'good', 'sentiment': 0.7}]


## aspect-based-sentiment-analysis 2.0.3
* can only work on google colab
* [python package](https://pypi.org/project/aspect-based-sentiment-analysis/)
* [example from youtube](https://www.youtube.com/watch?v=q8sTicXK4Fg)

## VaderSentiment.vaderSentiment
* [Kaggle notebook](https://www.kaggle.com/code/phiitm/aspect-based-sentiment-analysis)

In [26]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#note: depending on how you installed (e.g., using source code download versus pip install), you may need to import like this:
#from vaderSentiment import SentimentIntensityAnalyzer

# --- examples -------
sentences = ["VADER is smart, handsome, and funny.",  # positive sentence example
             "VADER is smart, handsome, and funny!",  # punctuation emphasis handled correctly (sentiment intensity adjusted)
             "VADER is very smart, handsome, and funny.", # booster words handled correctly (sentiment intensity adjusted)
             "VADER is VERY SMART, handsome, and FUNNY.",  # emphasis for ALLCAPS handled
             "VADER is VERY SMART, handsome, and FUNNY!!!", # combination of signals - VADER appropriately adjusts intensity
             "VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!", # booster words & punctuation make this close to ceiling for score
             "VADER is not smart, handsome, nor funny.",  # negation sentence example
             "The book was good.",  # positive sentence
             "At least it isn't a horrible book.",  # negated negative sentence with contraction
             "The book was only kind of good.", # qualified positive sentence is handled correctly (intensity adjusted)
             "The plot was good, but the characters are uncompelling and the dialog is not great.", # mixed negation sentence
             "Today SUX!",  # negative slang with capitalization emphasis
             "Today only kinda sux! But I'll get by, lol", # mixed sentiment example with slang and constrastive conjunction "but"
             "Make sure you :) or :D today!",  # emoticons handled
             "Catch utf-8 emoji such as such as 💘 and 💋 and 😁",  # emojis handled
             "Not bad at all"  # Capitalized negation
             ]

analyzer = SentimentIntensityAnalyzer()
for sentence in sentences:
    vs = analyzer.polarity_scores(sentence)
    print("{:-<65} {}".format(sentence, str(vs)))

VADER is smart, handsome, and funny.----------------------------- {'neg': 0.0, 'neu': 0.254, 'pos': 0.746, 'compound': 0.8316}
VADER is smart, handsome, and funny!----------------------------- {'neg': 0.0, 'neu': 0.248, 'pos': 0.752, 'compound': 0.8439}
VADER is very smart, handsome, and funny.------------------------ {'neg': 0.0, 'neu': 0.299, 'pos': 0.701, 'compound': 0.8545}
VADER is VERY SMART, handsome, and FUNNY.------------------------ {'neg': 0.0, 'neu': 0.246, 'pos': 0.754, 'compound': 0.9227}
VADER is VERY SMART, handsome, and FUNNY!!!---------------------- {'neg': 0.0, 'neu': 0.233, 'pos': 0.767, 'compound': 0.9342}
VADER is VERY SMART, uber handsome, and FRIGGIN FUNNY!!!--------- {'neg': 0.0, 'neu': 0.294, 'pos': 0.706, 'compound': 0.9469}
VADER is not smart, handsome, nor funny.------------------------- {'neg': 0.646, 'neu': 0.354, 'pos': 0.0, 'compound': -0.7424}
The book was good.----------------------------------------------- {'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'co

## HuggingFace

In [27]:
import torch
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v2-xlarge", problem_type="multi_label_classification")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0) > 0.5]

# To train a model on `num_labels` classes, you can pass `num_labels=num_labels` to `.from_pretrained(...)`
num_labels = len(model.config.id2label)
model = DebertaV2ForSequenceClassification.from_pretrained(
    "microsoft/deberta-v2-xlarge", num_labels=num_labels, problem_type="multi_label_classification"
)

RuntimeError: Failed to import transformers.models.deberta_v2.modeling_deberta_v2 because of the following error (look up to see its traceback):
cannot import name 'dispatch_model' from 'accelerate' (/home/adam/.local/lib/python3.7/site-packages/accelerate/__init__.py)

In [28]:
import torch
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification

sentiment_labels = ["Positive", "Neutral","Negative"]
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v2-xlarge")
model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v2-xlarge")
model.eval()

input_text = "Hello, my dog is ugly"
inputs = tokenizer(input_text, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_ids = torch.arange(0, logits.shape[-1])[torch.sigmoid(logits).squeeze(dim=0)>0.5]
print(logits)

# Interpret the sentiment result
sentiment = sentiment_labels[predicted_class_ids]

print("Input Text: ", input_text)
print("Predicted Sentiment: ", sentiment)

RuntimeError: Failed to import transformers.models.deberta_v2.modeling_deberta_v2 because of the following error (look up to see its traceback):
cannot import name 'dispatch_model' from 'accelerate' (/home/adam/.local/lib/python3.7/site-packages/accelerate/__init__.py)

In [29]:
tmp = train_df[train_df.name == 'Cafe Patachou']

In [30]:
sentences = list(tmp.text)
sentences = list(dict.fromkeys(sentences)) # deduplicate

In [31]:
Counter(train_df.name)

Counter({'On The Border Mexican Grill & Cantina': 18,
         'China Kitchen': 16,
         "Landry's Seafood House": 15,
         'Mariscos Chihuahua': 16,
         'Burger Republic': 16,
         'Tandoor & Tikka': 16,
         "Giordano's": 16,
         'Pizzeria Vetri': 16,
         'Johnny Rockets': 6,
         'HipCityVeg': 15,
         'Ekta Indian Cuisine': 16,
         'Firebirds Wood Fired Grill': 19,
         "Dave & Buster's": 22,
         'Noble Crust': 16,
         'The Hungry Greek': 16,
         'Sauce on the Side': 18,
         "Carmelita's Mexican Restaurant": 16,
         'Han Dynasty': 18,
         "Joe's Crab Shack": 16,
         'Mamas Kitchen': 12,
         'Fellini Cafe': 16,
         'Earl of Sandwich': 17,
         'sweetgreen': 13,
         'honeygrow': 21,
         'Las Palmas Mexican Restaurant': 19,
         "BJ's Restaurant & Brewhouse": 20,
         'The Pasta House': 17,
         'Louis Pappas Fresh Greek': 16,
         'Tokyo Mandarin': 16,
         "

In [32]:
import torch
from transformers import DebertaV2ForSequenceClassification, DebertaV2Tokenizer

# Load the pretrained model and tokenizer
model_name = "microsoft/deberta-v2-xlarge"  # Pretrained DeBERTa-v2 model
tokenizer = DebertaV2Tokenizer.from_pretrained(model_name)
model = DebertaV2ForSequenceClassification.from_pretrained(model_name)

# Set the device for inference
model.to(device)
model.eval()

# Example input text and aspect
input_text = sentences[0]
aspect = "Food"

def prediction(input_text , aspect):
    # Combine the input text and aspect
    input_text_with_aspect = f"{input_text} [ASP] {aspect}"

    # Tokenize the input text with aspect
    encoded_input = tokenizer.encode_plus(input_text_with_aspect, padding=True, truncation=True, return_tensors="pt")
    input_ids = encoded_input["input_ids"].to(device)
    attention_mask = encoded_input["attention_mask"].to(device)

    # Perform aspect sentiment analysis inference
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Get the predicted sentiment label
    predicted_sentiment = torch.argmax(logits, dim=1).item()

    # Interpret the sentiment result
    sentiment_labels = ["Positive", "Neutral", "Negative"]
    sentiment = sentiment_labels[predicted_sentiment]

    print("Input Text: ", input_text)
    print("Aspect: ", aspect)
    print("Predicted Sentiment: ", sentiment)

RuntimeError: Failed to import transformers.models.deberta_v2.modeling_deberta_v2 because of the following error (look up to see its traceback):
cannot import name 'dispatch_model' from 'accelerate' (/home/adam/.local/lib/python3.7/site-packages/accelerate/__init__.py)

In [33]:
for i in sentences:
    prediction(i , 'food')

NameError: name 'prediction' is not defined